# Attempt 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset_file_path = '/content/drive/MyDrive/Exam_buddy_finetuning/final_tiny.csv'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(dataset_file_path)
df

,question,answer
0,b'Let m = -9537 - -9547. Suppose -6 = -5*s + 1...,b'20\n'
1,b'Let z = -36.6 + 57. Let m = 913 - 934. Let g...,b'-0.6\n'
2,b'What is -515384.2 times 5?\n',b'-2576921\n'
3,b'Let x(p) = -11*p**2 + 924*p - 8229. Give x(9...,b'-804\n'
4,b'Simplify (2*1*sqrt(85))/(sqrt(320)*-1) + -4.\n',b'-4 - sqrt(17)/4\n'
...,...,...
1135,b'Let y(b) = 8*b**2 - 834*b + 17458. Give y(29...,b'0\n'
1136,b'Simplify (r/((r/((r/r**(-18))/r))/r))/r*r**(...,b'r**(96/5)\n'
1137,b'What is prob of sequence pp when two letters...,b'22/57\n'
1138,"b'Suppose 3*i - 175*d = -170*d + 10247, -5*i -...",b'4\n'


In [5]:
!pip install datasets
from datasets import Dataset, DatasetDict

In [6]:
def create_dataset_hf(dataframe):
    dataset = Dataset.from_pandas(dataframe)
    return DatasetDict({"train": dataset})

math_dataset = create_dataset_hf(df)

In [7]:
HF_ACCESS_TOKEN="Mai_nahi_bataunga"

In [1]:
from huggingface_hub import notebook_login

notebook_login()

huggingface_user = "AiLover69"
dataset_name = "math_dataset_tiny"

math_dataset.push_to_hub(f"{huggingface_user}/{dataset_name}")


NameError: name 'math_dataset' is not defined

In [ ]:
pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121


ERROR: Could not find a version that satisfies the requirement xformers (from versions: none)
ERROR: No matching distribution found for xformers


^C


In [2]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig
from datasets import load_dataset
import logging

KeyboardInterrupt: 

In [1]:
import torch
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

ModuleNotFoundError: No module named 'trl'

In [14]:
# Load the pre-trained model and tokenizer
model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit"
logging.info(f"Loading model and tokenizer for: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [15]:
huggingface_user = "AiLover69"
dataset_name = "math_dataset_tiny"

In [16]:
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username":"Ailover69",
    "model_config": {
        "base_model":"unsloth/llama-3-8b-Instruct-bnb-4bit", # The base model
        "finetuned_model":"llama-3-8b-Instruct-bnb-4bit-aiaustin-demo", # The finetuned model
        "max_seq_length": 2048, # The maximum sequence length
        "dtype":torch.float16, # The data type
        "load_in_4bit": True, # Load the model in 4-bit
    },
    "lora_config": {
      "r": 16, # The number of LoRA layers 8, 16, 32, 64
      "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # The target modules
      "lora_alpha":16, # The alpha value for LoRA
      "lora_dropout":0, # The dropout value for LoRA
      "bias":"none", # The bias for LoRA
      "use_gradient_checkpointing":True, # Use gradient checkpointing
      "use_rslora":False, # Use RSLora
      "use_dora":False, # Use DoRa
      "loftq_config":None # The LoFTQ configuration
    },
     "training_dataset":{
        "name":f"{huggingface_user}/{dataset_name}", # The dataset name(huggingface/datasets)
        "split":"train", # The dataset split
        "input_field":"question", # The input field
    },
    "training_config": {
        "per_device_train_batch_size": 2, # The batch size
        "gradient_accumulation_steps": 4, # The gradient accumulation steps
        "warmup_steps": 5, # The warmup steps
        "max_steps":0, # The maximum steps (0 if the epochs are defined)
        "num_train_epochs": 10, # The number of training epochs(0 if the maximum steps are defined)
        "learning_rate": 2e-4, # The learning rate
        "fp16": torch.cuda.is_bf16_supported(), # The fp16
        "bf16": False, # The bf16
        "logging_steps": 1, # The logging steps
        "optim" :"adamw_8bit", # The optimizer
        "weight_decay" : 0.01,  # The weight decay
        "lr_scheduler_type": "linear", # The learning rate scheduler
        "seed" : 42, # The seed
        "output_dir" : "/content/drive/MyDrive/Exam_buddy_finetuning/outputs",  # The output directory", # The output directory
    }
}

In [17]:
import torch

# Loading the model and the tokenizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.get("model_config").get("base_model"),
    max_seq_length=config.get("model_config").get("max_seq_length"),
    dtype=torch.float16,  # Set data type to half-precision
    load_in_4bit=config.get("model_config").get("load_in_4bit"),
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r=config.get("lora_config").get("r"),
    target_modules=config.get("lora_config").get("target_modules"),
    lora_alpha=config.get("lora_config").get("lora_alpha"),
    lora_dropout=config.get("lora_config").get("lora_dropout"),
    bias=config.get("lora_config").get("bias"),
    use_gradient_checkpointing=config.get("lora_config").get("use_gradient_checkpointing"),
    random_state=42,
    use_rslora=config.get("lora_config").get("use_rslora"),
    use_dora=config.get("lora_config").get("use_dora"),
    loftq_config=config.get("lora_config").get("loftq_config"),
    use_gradient_checkpointing = "unsloth", # <<<<<<<
)

# Loading the training dataset
dataset_train = load_dataset(config.get("training_dataset").get("name"), split=config.get("training_dataset").get("split"))

# Setting up the trainer for the model


''' If there is a checkpoint file in drive and we want to resume it

# Check if a checkpoint exists and load it
checkpoint = config.get("training_config").get("output_dir")+"/checkpoint-1000"  # Replace 1000 with your checkpoint file digits
if os.path.exists(checkpoint):
    training_args.resume_from_checkpoint = checkpoint
'''

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    dataset_text_field=config.get("training_dataset").get("input_field"),
    max_seq_length=config.get("model_config").get("max_seq_length"),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps=config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps=config.get("training_config").get("warmup_steps"),
        max_steps=config.get("training_config").get("max_steps"),
        num_train_epochs=config.get("training_config").get("num_train_epochs"),
        learning_rate=config.get("training_config").get("learning_rate"),
        fp16=True,  # Use half-precision
        bf16=False,
        logging_steps=config.get("training_config").get("logging_steps"),
        optim=config.get("training_config").get("optim"),
        weight_decay=config.get("training_config").get("weight_decay"),
        lr_scheduler_type=config.get("training_config").get("lr_scheduler_type"),
        seed=42,
        output_dir=config.get("training_config").get("output_dir"),
        save_strategy="epoch", 
        save_steps=500, 
    ),
)


==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Generating train split:   0%|          | 0/1140 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/1140 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,140 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,420
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.466800
2,3.971500
3,4.014900
4,4.124300
5,3.988800
6,3.206400
7,3.323600
8,2.892800
9,3.028400
10,2.327300


## GPT Code for training

In [ ]:
# Define training arguments

# CURRENTLY NOT IN USE IGNORE IT
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=0,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=1,
    fp16=True,
    optim="adamw_8bit",
    lr_scheduler_type="linear",
    seed=42,
)


logging.info("Loading training dataset from Hugging Face hub.")
dataset_train = load_dataset("AiLover69/math_dataset", split='train')
print(dataset_train)

logging.info("Initializing the Trainer.")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    tokenizer=tokenizer,
)

# Fine-tune the model
logging.info("Starting model fine-tuning.")
trainer.train()
logging.info("Model fine-tuning completed.")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Dataset({
    features: ['question', 'answer'],
    num_rows: 1120000
})


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 0 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,400,000
 "-____-"     Number of trainable parameters = 41,943,040


IndexError: Invalid key: 649122 is out of bounds for size 0

##Inference Step

In [ ]:

# Configuration for the inference model
model_checkpoint = config.get("training_config").get("output_dir")+"/checkpoint-1000"  # Replace 1000 with your best checkpoint file digits & Path to your best checkpoint

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs["input_ids"], max_length=2048, do_sample=True, top_p=0.95, top_k=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
# Example usage
prompt = "Give the solution of Riemann hypothesis?"
response = generate_response(prompt)
print(response)